### **Ridefinizione del Progetto**

Il progetto si evolve in una pipeline duale che combina due approcci distinti per l'elaborazione di documenti scientifici su ArXiv, uno focalizzato sull'information retrieval multimodale per il QA e l'altro orientato alla creazione e utilizzo di un grafo per il reasoning avanzato. Ecco una sintesi della nuova struttura:

---

### **1. Obiettivo Generale**
Creare un Google Colab notebook che esplora due metodi distinti:
1. **Approccio Multimodale per il QA**: Retrieval di contenuti (testo e visivi) da documenti ArXiv con valutazione quantitativa delle prestazioni su un task di multiple-choice QA.
2. **GraphRAG per Reasoning Avanzato**: Creazione e utilizzo di un grafo di conoscenza a partire da documenti ArXiv per rispondere a domande complesse (multi-hop reasoning e aggregazioni) attraverso tecnologie avanzate.

---


### **2. Descrizione dei Due Moduli**

#### **Modulo 1: Approccio Multimodale per QA**
- **Focus**: Estrarre chunk di testo e figure dai PDF, utilizzarli in pipeline di retrieval (multimodale e text-only), e valutarli quantitativamente in un task di QA multiple-choice.
- **Tecnologie**:
  - Modelli multimodali (es. ColQwen2).
  - Late interaction per text-only retrieval.
  - Metriche di valutazione come Precision@k, MRR e accuratezza downstream su QA.
- **Output**:
  - Risultati numerici sulle prestazioni dei vari approcci.
  - Tabelle e grafici comparativi.

---

#### **Modulo 2: GraphRAG per Reasoning Avanzato**
- **Focus**: Costruire un grafo di conoscenza a partire dai documenti ArXiv (estrazione di entità e relazioni) per rispondere a domande complesse.
- **Pipeline**:
  1. **Estrazione di Entità e Relazioni**:
     - Uso di LLM (es. GPT-4o-mini) per identificare nodi e archi, con varianti per aggiungere forza relazionale o descrizioni.
  2. **Popolazione del Grafo**:
     - Archiviazione in un database Neo4j per interrogazioni successive.
  3. **Interrogazione del Grafo**:
     - Approcci:
       - Vicini diretti di entità.
       - Community detection e clustering.
       - Generazione di query Cypher con un LLM.
  4. **Reasoning e Risposta**:
     - Generazione di risposte condizionate da cluster di entità.
     - Combina risposte locali e globali con pesi stimati.
- **Tecnologie**:
  - **Neo4j** per graphDB.
  - **Milvus** per indicizzazione come VectorDB.
  - **LangGraph** per pipeline automatizzata.
- **Output**:
  - Demo interattiva con domande esempio.
  - Visualizzazioni di query sul grafo e risposte.

---

## Setup

In [ ]:
!pip install datasets
!pip install pillow
!pip install matplotlib
!pip install transformers torch datasets byaldi
!pip install -q pdf2image 
!pip install git+https://github.com/huggingface/transformers.git 
!pip install qwen-vl-utils 
!pip install flash-attn
!pip install byaldi

In [ ]:
!pip install torch torchvision

In [ ]:
!sudo apt-get install -y poppler-utils

[sudo] password for rgiordano: 


In [12]:
# Import required libraries
import os

from datasets import load_dataset

import torch
import transformers
from byaldi import RAGMultiModalModel
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

import tqdm

from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd

In [6]:
# Set device
device = "mps" if torch.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu"
    
print(f"Using device: {device}")

Using device: mps


## Load Dataset

In [7]:
"""
@misc{li2024multimodal,
            title={Multimodal ArXiv: A Dataset for Improving Scientific Comprehension of Large Vision-Language Models}, 
            author={Lei Li and Yuqi Wang and Runxin Xu and Peiyi Wang and Xiachong Feng and Lingpeng Kong and Qi Liu},
            year={2024},
            eprint={2403.00231},
            archivePrefix={arXiv},
            primaryClass={cs.CV}
        }
        """

# Load parquet files inside folder data
ds = load_dataset('parquet', data_files='data/*.parquet')

In [64]:
import requests

def download_arxiv_pdf(arxiv_id, save_path):
    url = f"https://arxiv.org/pdf/{arxiv_id}"
    response = requests.get(url)
    if response.status_code == 200:
        with open(save_path, 'wb') as file:
            file.write(response.content)
        print(f"PDF downloaded successfully: {save_path}")
    else:
        print(f"Failed to download PDF. Status code: {response.status_code}")

# Example usage
arxiv_id = "1810.10511"
save_path = f"{arxiv_id}.pdf"
download_arxiv_pdf(arxiv_id, save_path)


PDF downloaded successfully: 1810.10511.pdf


In [4]:
ds['train'][0]

{'query': 'Based on the graph, what is the impact of correcting for fspec not equal to 1 on the surface density trend?',
 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1621x1191>,
 'image_filename': 'images/1810.10511_2.jpg',
 'options': "['A. Correction causes a significant increase in surface density across all radii.', 'B. Correction results in a decrease in surface density for larger radii.', 'C. Correction causes the surface density to converge with the fspec = 1 case at larger radii.', 'D. Correction does not affect the surface density trend at all.', '-']",
 'answer': 'C',
 'page': '',
 'model': 'gpt4V',
 'prompt': '',
 'source': 'arxiv_qa'}

In [25]:
# Save all images
PATH='data/images'
os.makedirs(PATH, exist_ok=True)

for i, item in enumerate(ds['train']):
    item['image'].save(f'data/{item["image_filename"]}')

#### Generation of image's description

In [8]:
multimodal_model_name = "Qwen/Qwen2-VL-2B-Instruct"
multimodal_model = Qwen2VLForConditionalGeneration.from_pretrained(
                                                        multimodal_model_name,
                                                        trust_remote_code=True, 
                                                        torch_dtype=torch.bfloat16).to(device)
processor = AutoProcessor.from_pretrained(multimodal_model_name, trust_remote_code=True)

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  6.32it/s]


In [9]:
def create_messages(img):
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": img,
                    "max_pixels": 720**2,
                },
                {
                    "type": "text", 
                    "text": 
                        "Based on the image, provide a detailed scientific description of the graph."
                },
            ],
        }
    ]

    return messages


In [16]:
def invoke_generation(messages):

    print('Applying vision template...')
    # Apply a chat template to the messages without tokenizing and add a generation prompt
    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )

    print('Tokenizing text...')
    # Process vision information from the messages to get image and video inputs
    image_inputs, video_inputs = process_vision_info(messages)

    print('Preparing inputs...')
    # Prepare the inputs for the model by combining text, images, and videos
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",  # Return the inputs as PyTorch tensors
    )

    print('Moving inputs to device...')
    # Move the inputs to the specified device (e.g., GPU)
    inputs = inputs.to(device)

    print('Generating output...')
    # Generate output IDs from the model with a maximum of 500 new tokens
    generated_ids = multimodal_model.generate(**inputs, max_new_tokens=500)

    print('Decoding output...')
    # Trim the generated IDs to remove the input IDs from the beginning
    generated_ids_trimmed = [
        out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]

    print('Decoding text...')
    # Decode the trimmed generated IDs to get the output text
    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )

    print('Output:', output_text)
    print('\n\n ---------------------------------------- \n\n')
    return output_text

In [19]:
# Function to process a single example
def process_item(example):
    try:
        img = example['image']  # Extract the image
        messages = create_messages(img)  # Create messages
        output_text = invoke_generation(messages)  # Generate output text
        return {"image_filename": example["image_filename"], "output_text": output_text}
    except Exception as e:
        print(f"Error processing item: {e}")

# Function to save processed data as a checkpoint
def save_checkpoint(processed_data, checkpoint_file):
    df = pd.DataFrame(processed_data)
    df.to_parquet(checkpoint_file, index=False)
    print(f"Checkpoint saved: {len(processed_data)} items to {checkpoint_file}")

# Process dataset in batches with checkpoints
def process_dataset_with_checkpoints(ds, batch_size=10, output_file="processed_images.parquet"):
    train_data = ds['train']  # Access training data
    processed_items = []  # To store processed items
    checkpoint_file = output_file

    for idx, example in enumerate(tqdm.tqdm(train_data)):
        processed_item = process_item(example)
        processed_items.append(processed_item)

        print(processed_items)

        # Save every `batch_size` items or at the end
        if (idx + 1) % batch_size == 0 or (idx + 1) == len(train_data):
            save_checkpoint(processed_items, checkpoint_file)

    return processed_items

# Run the processing
processed_dataset = process_dataset_with_checkpoints(
    ds, batch_size=10, output_file="processed_images.parquet"
)


  0%|          | 0/500 [00:00<?, ?it/s]

Applying vision template...
Tokenizing text...
Preparing inputs...
Moving inputs to device...
Generating output...


  0%|          | 1/500 [00:27<3:52:47, 27.99s/it]

Decoding output...
Decoding text...
Output: ['The graph in the image is a plot of the surface density of a hypothetical object as a function of its radius. The x-axis represents the radius of the object in units of h^{-1} Mpc, while the y-axis represents the surface density in units of h^2 Mpc^{-2}. The surface density is calculated using the formula:\n\n\\[ \\Sigma = \\frac{1}{4\\pi R^2} \\]\n\nwhere \\( R \\) is the radius of the object.\n\nThere are three lines on the graph:\n\n1. **Black line (f_spec = 1)**: This line represents the surface density when the factor \\( f_{\\text{spec}} \\) is equal to 1. This line is a straight line with a slope of -1, indicating that the surface density decreases as the radius increases.\n\n2. **Blue dashed line (f_spec ≠ 1, w/o corr.)**: This line represents the surface density when the factor \\( f_{\\text{spec}} \\) is not equal to 1, but without any correction. This line is a curved line with a slope that is not -1, indicating that the surface 

  0%|          | 2/500 [00:44<2:58:15, 21.48s/it]

...

Decoding output...
Decoding text...
Output: ['The graph in the image is a plot of \\( r_h / r_0 \\) against \\( \\tau / r_0 \\) for a Schwarzschild-AdS system. The x-axis represents \\( \\tau / r_0 \\), which is a dimensionless time variable, and the y-axis represents \\( r_h / r_0 \\), which is a dimensionless radius variable.\n\nThe graph shows a curve that starts at a point near the origin (0,0) and then decreases as \\( \\tau / r_0 \\) increases. The curve approaches a horizontal asymptote at \\( r_h / r_0 = 0 \\) as \\( \\tau / r_0 \\) approaches infinity. This indicates that as time \\( \\tau \\) becomes very large, the radius \\( r_h \\) approaches zero.\n\nThere are two vertical dashed lines on the graph:\n1. The first vertical dashed line is at \\( \\tau_1 / r_0 \\), which is a critical point on the curve. This point is where the curve starts to deviate from the horizontal asymptote.\n2. The second vertical dashed line is at \\( \\tau_c / r_0 \\), which is another critical poi

  6%|▌         | 29/500 [12:53<3:53:09, 29.70s/it]

Decoding output...
Decoding text...
Output: ["The image represents a mind map that illustrates the logical reasoning process involved in a medical scenario. The mind map is divided into four main sections: MedNLI, RadQA, CLIP, and LLM.\n\n1. **MedNLI (Medical NLI)**:\n   - **Premise**: The patient emerged with Apgar scores of 7 and 8.\n   - **Hypothesis**: The patient had low Apgar scores.\n   - **Contradiction**: The patient had high Apgar scores.\n\n2. **RadQA (Radiology QA)**:\n   - **Context**: The emergency room clinicians requested a second read on a C-spine CT.\n   - **Finding**: There is no evidence of evidence of fracture or subluxation.\n   - **Question**: Are there any abnormalities in the C-spine?\n\n3. **CLIP (Clinical Imaging Pathology)**:\n   - **Patient**: The patient has a follow-up neck CTA and appointment with a surgery on 1978-10-18.\n   - **Appointment-related, Imaging-related, Procedure-related follow-ups**.\n\nThe mind map shows that the patient's high Apgar scor

  6%|▌         | 30/500 [13:20<3:45:17, 28.76s/it]

Decoding output...
Decoding text...
Output: ['The image is a series of six panels, each depicting a different setup of a magnetic field configuration over time. The panels are labeled with the following:\n\n1. **Initial state (time = 0 h)**: This panel shows the initial state of the magnetic field configuration at the start of the simulation.\n2. **Setup (A) (time = 10 h)**: This panel shows the magnetic field configuration after 10 hours of simulation time.\n3. **Setup (B) (time = 10 h)**: This panel shows the magnetic field configuration after 10 hours of simulation time.\n4. **Setup (C) (time = 10 h)**: This panel shows the magnetic field configuration after 10 hours of simulation time.\n5. **Setup (D) (time = 10 h)**: This panel shows the magnetic field configuration after 10 hours of simulation time.\n6. **Setup (E) (time = 20 h)**: This panel shows the magnetic field configuration after 20 hours of simulation time.\n\nEach panel includes a color bar on the left side, indicating t

## 1. Multimodal Representational Models with Late Interaction

#### Load model colqwen2-v0.1

In [6]:
RAG = RAGMultiModalModel.from_pretrained("./models/colqwen2-v0.1-merged", device=device)

Verbosity is set to 1 (active). Pass verbose=0 to make quieter.


In [11]:
try:
    RAG.index(
    input_path="data/images/", # The path to your documents
    index_name='colqwen2-v0.1-merged-arxiv_qa', # The name you want to give to your index. It'll be saved at `index_root/index_name/`.
    store_collection_with_index=False, # Whether the index should store the base64 encoded documents.
    overwrite=False # Whether to overwrite an index if it already exists. If False, it'll return None and do nothing if `index_root/index_name` exists.
)
except ValueError:
    RAG=RAG.from_index('colqwen2-v0.1-merged-arxiv_qa', device='cpu')

Verbosity is set to 1 (active). Pass verbose=0 to make quieter.


/Users/robertogiordano/Desktop/MultiGraphQA/.conda/lib/python3.11/site-packages/byaldi/colpali.py:162: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.indexed_embeddings.

In [14]:
text_query = 'Based on the graph, what is the impact of correcting for fspec not equal to 1 on the surface density trend?'
results = RAG.search(text_query, k=1)
results

[{'doc_id': 245, 'page_num': 1, 'score': 24.875, 'metadata': {}, 'base64': None}]

In [29]:
import gzip
import json

# Open the .json.gz file
with gzip.open('.byaldi/colqwen2-v0.1-merged-arxiv_qa/index_config.json.gz', 'rt', encoding='utf-8') as f:
    data = json.load(f)

# Print or process the JSON data
print(data)


{'model_name': './models/colqwen2-v0.1-merged', 'full_document_collection': False, 'highest_doc_id': 499, 'resize_stored_images': False, 'max_image_width': None, 'max_image_height': None, 'library_version': '0.0.7'}


In [27]:
image_name='/'.join(RAG.get_doc_ids_to_file_names()[results[0]['doc_id']].split('/')[-2:])

In [ ]:
del RAG

NameError: name 'RAG' is not defined

In [53]:
torch.mps.empty_cache()

In [42]:
# Find image_name in ds
for i, item in enumerate(ds['train']):
    if item['image_filename']==image_name:
        break

#### RAG Pipeline Qwen

In [18]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch

model_name = "Qwen/Qwen2-VL-2B-Instruct"
model = Qwen2VLForConditionalGeneration.from_pretrained(model_name,
                                                        trust_remote_code=True, torch_dtype=torch.bfloat16).to(device)

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  4.77it/s]


In [7]:
item=ds['train'][0]

In [27]:
item

{'query': 'Based on the graph, what is the impact of correcting for fspec not equal to 1 on the surface density trend?',
 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1621x1191>,
 'image_filename': 'images/1810.10511_2.jpg',
 'options': "['A. Correction causes a significant increase in surface density across all radii.', 'B. Correction results in a decrease in surface density for larger radii.', 'C. Correction causes the surface density to converge with the fspec = 1 case at larger radii.', 'D. Correction does not affect the surface density trend at all.', '-']",
 'answer': 'C',
 'page': '',
 'model': 'gpt4V',
 'prompt': '',
 'source': 'arxiv_qa'}

In [65]:
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct", trust_remote_code=True)

# messages = [
#     {
#         "role": "user",
#         "content": [
#             {
#                 "type": "image",
#                 "image": item['image'],
#                 "max_pixels": 720**2,
#             },
#             {
#                 "type": "text", 
#                 "text": 
#                     item['query'] + 
#                     "\n  Choose the correct answer from the options below: \n" +
#                     item['options'] +
#                     "Answer with the letter of the correct option."

#             },
#         ],
#     }
# ]

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": item['image'],
                "max_pixels": 720**2,
            },
            {
                "type": "text", 
                "text": 
                    "Based on the image, provide a detailed scientific description of the graph."

            },
        ],
    }
]

In [66]:
print(messages)

[{'role': 'user', 'content': [{'type': 'image', 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1621x1191 at 0x3B61D7C90>, 'max_pixels': 518400}, {'type': 'text', 'text': 'Based on the image, provide a detailed scientific description of the graph.'}]}]


In [67]:
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

In [68]:
print(text)

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
<|vision_start|><|image_pad|><|vision_end|>Based on the image, provide a detailed scientific description of the graph.<|im_end|>
<|im_start|>assistant



In [69]:
image_inputs, video_inputs = process_vision_info(messages)

In [70]:
image_inputs

[<PIL.Image.Image image mode=RGB size=812x616>]

In [71]:
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to(device)

In [75]:
generated_ids = model.generate(**inputs, max_new_tokens=500)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)


In [76]:
print(output_text[0])

The graph in the image is a plot of the surface density of a hypothetical object as a function of its radius. The x-axis represents the radius of the object in units of h^{-1} Mpc, while the y-axis represents the surface density in units of h^2 Mpc^{-2}. The surface density is calculated using the formula:

\[ \Sigma = \frac{1}{4\pi R^2} \]

where \( R \) is the radius of the object.

There are three lines on the graph:

1. **Black line (f_spec = 1)**: This line represents the surface density when the factor \( f_{\text{spec}} \) is equal to 1. This line is a straight line with a slope of -1, indicating that the surface density decreases as the radius increases.

2. **Blue dashed line (f_spec ≠ 1, w/o corr.)**: This line represents the surface density when the factor \( f_{\text{spec}} \) is not equal to 1, but without any correction. This line is a curved line with a slope that is not -1, indicating that the surface density decreases more steeply as the radius increases.

3. **Blue so

#### RAG pipeline

In [5]:
# model_id = "google/gemma-2-2b-it"
model_id = "models/gemma-2-2b-it"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device=device
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  4.64it/s]


In [6]:
messages = [
    {"role": "user", "content": "Who are you? Please, answer in pirate-speak."},
]

outputs = pipeline(messages, max_new_tokens=256)
assistant_response = outputs[0]["generated_text"][-1]["content"].strip()
print(assistant_response)
# Ahoy, matey! I be Gemma, a digital scallywag, a language-slingin' parrot of the digital seas. I be here to help ye with yer wordy woes, answer yer questions, and spin ye yarns of the digital world.  So, what be yer pleasure, eh? 🦜

Ahoy, matey! I be Gemma, a digital scallywag, a language-slingin' parrot of the digital seas.  I be here to help ye with yer wordy woes, answer yer questions, and spin ye yarns of the digital world.  So, what be yer pleasure, eh? 🦜


#### **Parte 1: Multimodal Retrieval**


3. **Implementazione Approcci**:
   - Multimodale con modelli avanzati.
   - Text-only con late interaction e late chunking.

4. **Pipeline di QA**: Risoluzione multiple-choice con un modello generativo.

5. **Valutazione**:
   - Metriche di retrieval.
   - Accuratezza su QA.

6. **Visualizzazione**: Risultati e grafici comparativi.

#### **Parte 2: GraphRAG**
1. **Setup**: Installazione librerie (Neo4j, Milvus, LangGraph).
2. **Estrazione Triple**:
   - LLM per nodi e archi.
   - Scelta tra aggiunta di forza relazionale o descrizioni.
3. **Costruzione del Grafo**:
   - Inserimento in Neo4j.
4. **Interrogazione e Reasoning**:
   - Community detection, vicini diretti, query Cypher.
   - Generazione risposte locali e globali.
5. **Demo Interattiva**:
   - Domande multi-hop con risposte strutturate.
6. **Visualizzazione**:
   - Visualizzazione del grafo e delle risposte.

---


### **4. Output Finale**
- **Modulo 1**: Report quantitativo sulle prestazioni dei modelli di retrieval nel task multimodale.
- **Modulo 2**: Grafo di conoscenza interattivo con risposte a domande complessive e reasoning complesso.